In [1]:
import torch
import itertools
from sklearn import metrics
import math
import time
import csv
import numpy as np
from scipy.optimize import linear_sum_assignment

In [2]:
def accuracy(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # Find optimal one-to-one mapping between cluster labels and true labels
    row_ind, col_ind = linear_sum_assignment(-contingency_matrix)
    return contingency_matrix[row_ind, col_ind].sum() / np.sum(contingency_matrix)

In [3]:
def GMM_without_Outlier(N,p,m,sigma,w):
    torch.manual_seed(0)
    count = torch.distributions.multinomial.Multinomial(N, w).sample()
    count = count.type(torch.int64)
    y=torch.zeros((N,1))
    x=torch.zeros((N,p))
    for i in range(m):
        pos = torch.randn(1, p)
        obs = pos + std[i]*torch.randn(int(count[i]),p)
        x[sum(count[:i]):sum(count[:i+1]),:]=obs
        y[sum(count[:i]):sum(count[:i+1]),:]=i;
    return x,y

In [4]:
def GMM_with_Outlier(N,p,m,sigma,w):
    torch.manual_seed(0) # we use seed=0 for all the bound tests except N vs m we use seed = 1
    count = torch.distributions.multinomial.Multinomial(N, w).sample()
    count = count.type(torch.int64)
    y=torch.zeros((N,1))
    x=torch.zeros((N,p))
    for i in range(m):
        pos = torch.randn(1, p)
        obs = pos + std[i]*torch.randn(int(count[i]),p)
        x[sum(count[:i]):sum(count[:i+1]),:]=obs
        y[sum(count[:i]):sum(count[:i+1]),:]=i;
    neg = torch.randn(int(count[m]), p)
    x[sum(count[:m]):sum(count[:m+1]),:]=neg
    y[sum(count[:m]):sum(count[:m+1]),:]=-1
    return x,y

In [5]:
def scrlm(x,n,m,rho,F,device,seed):
    torch.manual_seed(seed)
    thr=0
    [N,p]=x.shape
    nsub = torch.randperm(N)
    nsub = nsub[:n]
    sloss=torch.zeros(n)
    sloss=sloss.to(device)
    loss=torch.cdist(x[nsub,:], x)
    loss=(loss**2)/(p*rho**2)-F
    loss[loss > 0] = 0
    sloss+=torch.sum(loss,1)
    idx = torch.argsort(sloss)
    idx = idx[sloss[idx]<-F]
    counter=0;
    sel= []
    while len(idx) and counter<=m-1:
        i=idx[0]
        sel.append(i)
        a = x[nsub[idx],:]
        b = x[nsub[i],:]
        dist = torch.sqrt(((a-b)**2).sum(axis=1))
        dist = (dist**2)/(p*rho**2)-F
        dist[dist > 0] = 0
        idx = idx[dist>=thr]
        counter=counter+1;
    result = torch.stack(sel)
    centers=x[nsub[result],:]
    return centers

In [6]:
N=20000
m=4
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
rho=0.5
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(20,40))
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
20 92
21 93
22 93
23 94
24 95
25 97
26 99
27 99
28 99
29 99
30 99
31 99
32 99
33 99
34 99
35 99
36 99
37 100
38 100
39 100


In [7]:
m=8
a = 0.7
w_min=a/m
w_max=0.9/m
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m_new.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(60,70))
    #n_list=[55]
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.)
torch.Size([20000, 1250])
60 97
61 97
62 97
63 97
64 98
65 99
66 99
67 100
68 100
69 100


In [8]:
N=20000
m=16
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
rho=0.5
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m_new.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(130,145))
    
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
130 98
131 98
132 98
133 99
134 99
135 99
136 99
137 99
138 99
139 99
140 99
141 99
142 99
143 99
144 100


In [9]:
N=20000
m=32
#n = 359=100
a = 0.7
p = 1250
w_min=a/m
w_max=0.9/m
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(350,360))
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
350 97
351 97
352 97
353 97
354 98
355 99
356 99
357 99
358 99
359 100


In [11]:
N=20000
m=64
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(610,622))
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
610 98
611 98
612 99
613 99
614 99
615 99
616 99
617 99
618 99
619 99
620 100
621 100


In [12]:
N=20000
m=128
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(1361,1487,5))
    print(n_list)
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.)
torch.Size([20000, 1250])
[1361, 1366, 1371, 1376, 1381, 1386, 1391, 1396, 1401, 1406, 1411, 1416, 1421, 1426, 1431, 1436, 1441, 1446, 1451, 1456, 1461, 1466, 1471, 1476, 1481, 1486]
1361 98
1366 99
1371 99
1376 99
1381 99
1386 99
1391 99
1396 99
1401 99
1406 99
1411 99
1416 99
1421 99
1426 99
1431 99
1436 99
1441 99
1446 99
1451 99
1456 99
1461 99
1466 99
1471 99
1476 99
1481 99
1486 100


In [13]:
N=20000
m=256
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(3828,3905,5))
    print(n_list)
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
[3828, 3833, 3838, 3843, 3848, 3853, 3858, 3863, 3868, 3873, 3878, 3883, 3888, 3893, 3898, 3903]
3828 98
3833 99
3838 99
3843 99
3848 99
3853 99
3858 99
3863 99
3868 99
3873 99
3878 99
3883 99
3888 99
3893 99
3898 100
3903 100


In [14]:
N=20000
m=512
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(7629,8410,10))
    print(n_list)
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
[7629, 7639, 7649, 7659, 7669, 7679, 7689, 7699, 7709, 7719, 7729, 7739, 7749, 7759, 7769, 7779, 7789, 7799, 7809, 7819, 7829, 7839, 7849, 7859, 7869, 7879, 7889, 7899, 7909, 7919, 7929, 7939, 7949, 7959, 7969, 7979, 7989, 7999, 8009, 8019, 8029, 8039, 8049, 8059, 8069, 8079, 8089, 8099, 8109, 8119, 8129, 8139, 8149, 8159, 8169, 8179, 8189, 8199, 8209, 8219, 8229, 8239, 8249, 8259, 8269, 8279, 8289, 8299, 8309, 8319, 8329, 8339, 8349, 8359, 8369, 8379, 8389, 8399, 8409]
7629 98
7639 99
7649 99
7659 99
7669 99
7679 99
7689 99
7699 99
7709 99
7719 99
7729 99
7739 99
7749 99
7759 99
7769 99
7779 99
7789 99
7799 99
7809 99
7819 99
7829 99
7839 99
7849 99
7859 99
7869 99
7879 99
7889 99
7899 99
7909 99
7919 99
7929 99
7939 99
7949 99
7959 99
7969 99
7979 99
7989 99
7999 99
8009 99
8019 99
8029 99
8039 99
8049 99
8059 99
8069 99
8079 99
8089 99
8099 99
8109 99
8119 99
8129 99
8139 99
8149 99
8159 99
8169 99
8179 99
8189 99
8199 99
8209 99
8219 99
8229

In [15]:
N=20000
m=1024
a = 0.7
w_min=a/m
w_max=0.9/m
p = 1250
run = 100
F=8/3
std=torch.linspace(1/16,0.25,m)
w=torch.linspace(w_min,w_max,m)
i = torch.tensor([0.2])
w = torch.cat((w, i), 0)
print(sum(w))
x,y = GMM_with_Outlier(N,p,m,std,w)
print(x.size())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x=x.to(device)
y=y.to(device)
with open('GMM_bound_n_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','p','m','a','rho','F','seed','accuracy','time','n','count','n_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'N':N, 'p':p,'m':m,'a':a,'rho':rho,'F':F})
    n_list=list(range(13608,14470,10))
    print(n_list)
    for k in range(len(n_list)):
        count = 0
        n = n_list[k]
        run =100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'n': n,'count':count})
        print(n,count)
        if count==100:
            writer.writerow({'n_':n,'count_':count})

tensor(1.0000)
torch.Size([20000, 1250])
[13608, 13618, 13628, 13638, 13648, 13658, 13668, 13678, 13688, 13698, 13708, 13718, 13728, 13738, 13748, 13758, 13768, 13778, 13788, 13798, 13808, 13818, 13828, 13838, 13848, 13858, 13868, 13878, 13888, 13898, 13908, 13918, 13928, 13938, 13948, 13958, 13968, 13978, 13988, 13998, 14008, 14018, 14028, 14038, 14048, 14058, 14068, 14078, 14088, 14098, 14108, 14118, 14128, 14138, 14148, 14158, 14168, 14178, 14188, 14198, 14208, 14218, 14228, 14238, 14248, 14258, 14268, 14278, 14288, 14298, 14308, 14318, 14328, 14338, 14348, 14358, 14368, 14378, 14388, 14398, 14408, 14418, 14428, 14438, 14448, 14458, 14468]
13608 98
13618 99
13628 99
13638 99
13648 99
13658 99
13668 99
13678 99
13688 99
13698 99
13708 99
13718 99
13728 99
13738 99
13748 99
13758 99
13768 99
13778 99
13788 99
13798 99
13808 99
13818 99
13828 99
13838 99
13848 99
13858 99
13868 99
13878 99
13888 99
13898 99
13908 99
13918 99
13928 99
13938 99
13948 99
13958 99
13968 99
13978 99
13988 9

In [16]:
with open('GMM_bound_p_vs_N.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    p_list=list(range(10,50))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**5
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

10 100
11 0
12 100
13 100
14 100
15 100
16 100
17 100
18 100
19 100
20 100
21 100
22 100
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100


In [17]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(10,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**6
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

10 100
11 3
12 100
13 100
14 100
15 94
16 100
17 0
18 0
19 0
20 100
21 100
22 69
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [18]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(10,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**7
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

10 0
11 100
12 100
13 60
14 0
15 100
16 100
17 100
18 0
19 0
20 100
21 100
22 100
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [19]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(10,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**8
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

10 11
11 3
12 0
13 1
14 100
15 23
16 0
17 100
18 30
19 0
20 0
21 0
22 100
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [20]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(20,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**9
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

20 61
21 100
22 99
23 100
24 100
25 93
26 100
27 100
28 91
29 100
30 100
31 100
32 100
33 100
34 60
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [21]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(20,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**10
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

20 0
21 0
22 43
23 0
24 0
25 72
26 100
27 0
28 100
29 100
30 49
31 0
32 55
33 60
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [22]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**11
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 100
31 0
32 17
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [23]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**12
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 0
31 87
32 100
33 79
34 100
35 97
36 100
37 77
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [24]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**13
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 1
31 90
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 97
40 85
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [25]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**14
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 51
31 99
32 100
33 100
34 96
35 88
36 32
37 100
38 100
39 73
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [26]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**15
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 94
31 0
32 0
33 1
34 0
35 51
36 100
37 82
38 0
39 0
40 9
41 0
42 0
43 0
44 70
45 100
46 99
47 4
48 100
49 100
50 100
51 98
52 100
53 100
54 100
55 100
56 100
57 100
58 99
59 100


In [27]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**16
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 20
31 100
32 0
33 0
34 2
35 3
36 0
37 45
38 81
39 100
40 96
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 81
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [28]:
with open('GMM_bound_p_vs_N.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(30,60))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=2**17
        m=3
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

30 0
31 0
32 0
33 51
34 98
35 100
36 0
37 83
38 100
39 0
40 46
41 100
42 4
43 99
44 100
45 0
46 0
47 100
48 100
49 100
50 100
51 100
52 70
53 100
54 100
55 100
56 100
57 100
58 100
59 100


In [55]:
with open('GMM_bound_N_vs_m_.csv', 'w', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    N_list=list(range(20,21))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 2
        p = 1250
        a = 0.7
        w_min = a/m
        w_max = 0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(5/0.01)))
        print(n)
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

20
20 100


In [77]:
with open('GMM_bound_N_vs_m_.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(18905,18906))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 1024
        p = 1250
        a = 0.7
        w_min = a/m
        w_max = 0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

18905
18905 100


In [40]:
with open('GMM_bound_p_vs_m.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    p_list=list(range(50,80))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=2
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

50 97
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100


In [41]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(40,70))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=4
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

40 100
41 41
42 94
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100


In [42]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(40,70))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=8
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

40 20
41 100
42 99
43 97
44 99
45 99
46 100
47 98
48 100
49 100
50 100
51 100
52 100
53 100
54 98
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100


In [43]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(40,70))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=16
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

40 22
41 92
42 61
43 100
44 38
45 0
46 91
47 12
48 70
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100


In [44]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(40,70))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=32
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

40 95
41 0
42 12
43 78
44 6
45 100
46 100
47 16
48 86
49 99
50 68
51 95
52 75
53 74
54 97
55 100
56 100
57 100
58 89
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100


In [45]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(40,80))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=64
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

40 0
41 0
42 3
43 57
44 24
45 4
46 24
47 80
48 35
49 99
50 98
51 94
52 100
53 30
54 75
55 74
56 12
57 100
58 38
59 32
60 100
61 100
62 100
63 100
64 100
65 97
66 100
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100


In [46]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(60,100))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=128
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

60 72
61 30
62 100
63 91
64 100
65 100
66 7
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100
80 100
81 100
82 100
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100


In [47]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(60,100))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=256
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

60 83
61 100
62 100
63 92
64 100
65 100
66 100
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100
80 100
81 100
82 100
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100


In [48]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(60,100))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=512
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

60 6
61 100
62 73
63 62
64 35
65 100
66 84
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 98
75 100
76 91
77 100
78 100
79 100
80 100
81 100
82 100
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100


In [49]:
with open('GMM_bound_p_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','rho','F','seed','accuracy','time','p','count','p_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    p_list=list(range(60,100))
    for k in range(len(p_list)):
        count = 0
        p = p_list[k]
        N=20000
        m=1024
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'p': p,'count':count})
        print(p,count)
        if count==100:
            writer.writerow({'p_':p,'count_':count})

60 94
61 100
62 0
63 100
64 100
65 100
66 100
67 0
68 100
69 6
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100
80 100
81 100
82 0
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100


In [86]:
with open('GMM_bound_rho_vs_m.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=2
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 88
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [87]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=4
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 94
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [88]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=8
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 99
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [89]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=16
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [90]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=32
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [91]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=64
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [92]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=128
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 2
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [93]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=256
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 5
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [94]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=512
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 1
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [95]:
with open('GMM_bound_rho_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=1024
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [96]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=32
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 100
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999

In [97]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=64
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [98]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**7
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 3
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [99]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**8
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 12
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.6799999

In [100]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**9
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [101]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**10
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 1
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [102]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**11
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [103]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**12
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 98
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [104]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**13
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 97
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [105]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**14
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 89
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [106]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**15
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 84
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [107]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**16
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 81
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [108]:
with open('GMM_bound_rho_vs_N_.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=2**17
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 70
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [6]:
with open('GMM_bound_rho_vs_p.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 2**11
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [7]:
with open('GMM_bound_rho_vs_p.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 2**12
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [8]:
with open('GMM_bound_rho_vs_p.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 2**13
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 17
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.6799999

In [9]:
with open('GMM_bound_rho_vs_p.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 2**14
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 91
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.6799999

In [10]:
with open('GMM_bound_rho_vs_p.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count','rho_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 2**15
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)
        if count==100:
            writer.writerow({'rho_':p,'count_':count})

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 100
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999

In [11]:
with open('GMM_bound_rho_vs_n.csv', 'w', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 32
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 95
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [12]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 64
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [13]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 128
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 99
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.679999999

In [14]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 256
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [15]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 512
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [16]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 1024
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [17]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 2048
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [18]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 4096
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [19]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 8192
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999

In [20]:
with open('GMM_bound_rho_vs_n.csv', 'a', newline='') as csvfile:
    fieldnames = ['N','n','m','a','p','F','seed','accuracy','time','rho','count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rho_list=np.linspace(0.2,0.9,71)
    print(rho_list)
    for k in range(len(rho_list)):
        count = 0
        rho = rho_list[k]
        N=20000
        m=3
        p = 1250
        a = 0.7
        w_min=a/m
        w_max=0.9/m
        n = 16384
        run = 100
        F=8/3
        std=torch.linspace(1/16,0.25,m)
        i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        w = torch.cat((w, i), 0)
        x,y = GMM_with_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        writer.writerow({'N':N, 'n':n,'m':m,'a':a,'rho':rho,'F':F})
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'rho': rho,'count':count})
        print(rho,count)

[0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33
 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47
 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61
 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75
 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89
 0.9 ]
0.2 0
0.21000000000000002 0
0.22 0
0.23 100
0.24000000000000002 100
0.25 100
0.26 100
0.27 100
0.28 100
0.29000000000000004 100
0.30000000000000004 100
0.31 100
0.32 100
0.33 100
0.34 100
0.35 100
0.36 100
0.37 100
0.38 100
0.39 100
0.4 100
0.41000000000000003 100
0.42000000000000004 100
0.43000000000000005 100
0.44 100
0.45 100
0.46 100
0.47000000000000003 100
0.48000000000000004 100
0.49 100
0.5 100
0.51 100
0.52 100
0.53 100
0.54 100
0.55 100
0.56 100
0.5700000000000001 100
0.5800000000000001 100
0.5900000000000001 100
0.6000000000000001 100
0.6100000000000001 100
0.62 100
0.63 100
0.64 100
0.65 100
0.66 100
0.67 100
0.67999999